<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Expert%20Mode/Face_Recognition_Expert_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.6.0


# Preparing Dataset
  

In [3]:
dataset_path = '/content/drive/MyDrive/7-7 dataset'

In [4]:
data_generation = ImageDataGenerator(rescale = 1./255,
                                     validation_split= 0.2,
                                    fill_mode = 'nearst')

width = height = 178
batch_size = 32

In [5]:
train_data = data_generation.flow_from_directory(
    dataset_path,
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    subset= 'training'
)

val_data = data_generation.flow_from_directory(
    dataset_path,
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
    subset= 'validation'
)

Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


# Define Model

In [65]:
class MyModel(tf.keras.Model):

  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),activation='relu', input_shape = (width ,height ,3))
    self.conv2=Conv2D(64,(3,3),activation='relu',)
    self.max1  = MaxPooling2D((2,2))
    self.conv3=Conv2D(128,(3,3),activation='relu')

    self.flatten=Flatten()
    self.fc1=Dense(128,activation='relu')
    self.fc2=Dense(number_of_classes, activation='sigmoid')

  def call(self,x):

    y = self.conv1(x)

    v = self.conv2(y)

    m = self.max1(v)

    p = self.conv3(m)

    w = self.flatten(p)

    z = self.fc1(w)

    output = self.fc2(z)
     
    return output
  
model = MyModel(14)

In [66]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [67]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name = 'train_loss')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_acc = tf.keras.metrics.CategoricalAccuracy(name = 'test_loss')

In [68]:
def train_step(images, y):
  with tf.GradientTape() as tape:
    y_pred = model(images, training=True)
    loss = loss_function(y, y_pred)

  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(y,y_pred)

In [69]:
def test_step(images, y):
  y_pred = model(images, training=False)
  loss = loss_function(y, y_pred)

  test_loss(loss)
  test_acc(y, y_pred)

In [70]:
from tqdm import tqdm

In [71]:
def train():
    epochs = 30
    c1 = 0
    c2 = 0
    for epoch  in range(epochs):
        train_loss.reset_state()
        train_acc.reset_state()

        test_loss.reset_state()
        test_acc.reset_state()
        
        for images,labels in train_data:
            train_step(images, labels)
            c1 += 1
            if c1 > 39:
                break

        for images,labels in val_data:
            test_step(images, labels) 
            c2 += 1
            if c2 > 39:
                break

        print('epoch:', epoch + 1)
        print('loss:', train_loss.result())
        print('accuracy:', train_acc.result())
        print('val loss:', test_loss.result())
        print('val accuracy:', test_acc.result())

# Fit

In [72]:
train()

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


epoch: 1
loss: tf.Tensor(4.5147777, shape=(), dtype=float32)
accuracy: tf.Tensor(0.2693845, shape=(), dtype=float32)
val loss: tf.Tensor(1.6529385, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.45083332, shape=(), dtype=float32)
epoch: 2
loss: tf.Tensor(1.4856656, shape=(), dtype=float32)
accuracy: tf.Tensor(0.53125, shape=(), dtype=float32)
val loss: tf.Tensor(1.476138, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.5833333, shape=(), dtype=float32)
epoch: 3
loss: tf.Tensor(1.4266913, shape=(), dtype=float32)
accuracy: tf.Tensor(0.5625, shape=(), dtype=float32)
val loss: tf.Tensor(1.0190156, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.625, shape=(), dtype=float32)
epoch: 4
loss: tf.Tensor(1.1716357, shape=(), dtype=float32)
accuracy: tf.Tensor(0.59375, shape=(), dtype=float32)
val loss: tf.Tensor(1.532099, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.40625, shape=(), dtype=float32)
epoch: 5
loss: tf.Tensor(1.5637476, shape=(), dtype=float32)
accuracy: tf.Tensor(0